In [17]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import importlib.util
import sys
spec = importlib.util.spec_from_file_location("preprocess", "../src/data/preprocess.py")
preprocess = importlib.util.module_from_spec(spec)
sys.modules["preprocess"] = preprocess
spec.loader.exec_module(preprocess)

spec = importlib.util.spec_from_file_location("module.name", "../src/features/build_features.py")
build_features = importlib.util.module_from_spec(spec)
sys.modules["build_features"] = build_features
spec.loader.exec_module(build_features)

spec = importlib.util.spec_from_file_location("train_model", "../src/models/train_model.py")
train_model = importlib.util.module_from_spec(spec)
sys.modules["train_model"] = train_model
spec.loader.exec_module(train_model)

spec = importlib.util.spec_from_file_location("predict_model", "../src/models/predict_model.py")
predict_model = importlib.util.module_from_spec(spec)
sys.modules["predict_model"] = predict_model
spec.loader.exec_module(predict_model)


Preprocessing и feature engeneering расписаны в скриптах "../src/data/preprocess.py" и "../src/features/build_features.py" соответственно

Итоговое решение сохранено в "../data/raw/sample_solution.csv"

In [14]:
train = pd.read_csv('../data/raw/train.csv')
x_test = pd.read_csv('../data/raw/test.csv')

In sex there is only 1 missed value, fill it with value thet matches with expression in corresponding function.

For the rest columnst missed values will filled with mean.

For columns 'Частота пасс кур' which type is object, process new feature instead of this - 'Пасс кур в неделю' which will be int:

'1-2 раза в неделю'   -->   ~2 times a week

'3-6 раз в неделю'   -->   ~4 times a week

'не менее 1 раза в день'   -->   ~10 times a week

'4 и более раз в день'   -->   ~35 times a week

'2-3 раза в день'   -->    ~25 times a week

I also mentioned, that if person has 0 in 'Пассивное курение' columns, he has NaN in 'Частота пасс кур'. That means that we can easily replace all NaNs in 'Частота пасс кур' with zero if in 'Пассивное курение' person has zero too. In whole dataset there is only one datapoint that does not fits this rule and has 'Пассивное курение' = 0 and 'Частота пасс кур' = '3-6 раз в неделю', so we have to fix it manually.

Same thing can be noticed in columns 'Возраст курения', 'Сигарет в день', 'Алкоголь', 'Возраст алког'.

In [15]:
TARGET_COLS = ['Артериальная гипертензия', 'ОНМК', 'Стенокардия, ИБС, инфаркт миокарда', 
               'Сердечная недостаточность', 'Прочие заболевания сердца']
x_train = train.drop(TARGET_COLS, axis=1)
y = train[TARGET_COLS]

Я пытался сделать нижеописанный пайплайн, написал свои трансформеры, но он не срабатывал по непонятной мне причине, что бы я ни менял(( 

Либо "A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.", либо "PreprocessTransformer.fit() takes 1 positional arguments but 2 were given".

При этом всём если вызывать трансформеры отдельно, то они работают и данные обрабатывают.

In [16]:
# pipe_of_insight = Pipeline([
#     ('cleaning', preprocess.PreprocessTransformer()),
#     ('features', build_features.FeaturesTransformer()),
#     ('catmodel', model)
# ])
transformer1 = build_features.FeaturesTransformer()
transformer1.fit(x_train, x_test)
x_train, x_test = transformer1.transform()

transformer2 = preprocess.PreprocessTransformer()
transformer2.fit(x_train, x_test)
x_train, x_test = transformer2.transform()

In [5]:
# Beacause the data is very imbalaanced, accuraacy score's value won't be much objective.
for col in y.columns:
    ones = sum(y[col] == 1)
    print('1 in y_data:', round(ones / y.shape[0] * 100, 2), '%')

1 in y_data: 46.7 %
1 in y_data: 4.29 %
1 in y_data: 12.25 %
1 in y_data: 10.05 %
1 in y_data: 9.01 %


Для оценки multilabel классификации я выбрал Micro Avg Precision/Recall. Она вычисляет confusion matrix для каждого класса и берёт среднее из всех наблюдений, вычисляя Precision и Recall. Таким образом учитывается неоднородность данных.

In [19]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y, random_state=64)
model = train_model.train(X_train, y_train)
precision, recall = predict_model.predict(model, X_train, X_test, y_train, y_test)

C:\Users\Andre\AppData\Local\Programs\Python\Python310\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


0:	learn: 0.5116319	total: 6.42ms	remaining: 636ms
1:	learn: 0.4403074	total: 13.2ms	remaining: 647ms
2:	learn: 0.3949237	total: 19.8ms	remaining: 641ms
3:	learn: 0.3643990	total: 26.9ms	remaining: 647ms
4:	learn: 0.3273531	total: 33ms	remaining: 627ms
5:	learn: 0.2694573	total: 38.7ms	remaining: 606ms
6:	learn: 0.2414559	total: 44.3ms	remaining: 589ms
7:	learn: 0.2153761	total: 49.5ms	remaining: 569ms
8:	learn: 0.1920776	total: 55.5ms	remaining: 562ms
9:	learn: 0.1611729	total: 60.8ms	remaining: 547ms
10:	learn: 0.1411663	total: 66.6ms	remaining: 539ms
11:	learn: 0.1198697	total: 71ms	remaining: 521ms
12:	learn: 0.1077713	total: 75.6ms	remaining: 506ms
13:	learn: 0.0930490	total: 80ms	remaining: 491ms
14:	learn: 0.0824950	total: 84.5ms	remaining: 479ms
15:	learn: 0.0666726	total: 89.3ms	remaining: 469ms
16:	learn: 0.0611359	total: 93.7ms	remaining: 457ms
17:	learn: 0.0585586	total: 97.9ms	remaining: 446ms
18:	learn: 0.0524180	total: 102ms	remaining: 436ms
19:	learn: 0.0503724	total: 1

87:	learn: 0.0009112	total: 437ms	remaining: 59.6ms
88:	learn: 0.0009016	total: 442ms	remaining: 54.6ms
89:	learn: 0.0008919	total: 447ms	remaining: 49.6ms
90:	learn: 0.0008853	total: 452ms	remaining: 44.7ms
91:	learn: 0.0008705	total: 457ms	remaining: 39.8ms
92:	learn: 0.0008648	total: 463ms	remaining: 34.8ms
93:	learn: 0.0008582	total: 467ms	remaining: 29.8ms
94:	learn: 0.0008533	total: 472ms	remaining: 24.8ms
95:	learn: 0.0008443	total: 477ms	remaining: 19.9ms
96:	learn: 0.0008356	total: 483ms	remaining: 14.9ms
97:	learn: 0.0008308	total: 488ms	remaining: 9.96ms
98:	learn: 0.0008237	total: 493ms	remaining: 4.98ms
99:	learn: 0.0008194	total: 498ms	remaining: 0us
0:	learn: 0.3298645	total: 4.84ms	remaining: 479ms
1:	learn: 0.2419024	total: 9.8ms	remaining: 480ms
2:	learn: 0.1925411	total: 14.5ms	remaining: 468ms
3:	learn: 0.1656146	total: 19.1ms	remaining: 459ms
4:	learn: 0.1239225	total: 24.2ms	remaining: 461ms
5:	learn: 0.0933855	total: 29.2ms	remaining: 457ms
6:	learn: 0.0817378	to

75:	learn: 0.0031242	total: 402ms	remaining: 127ms
76:	learn: 0.0030872	total: 408ms	remaining: 122ms
77:	learn: 0.0029849	total: 413ms	remaining: 117ms
78:	learn: 0.0029539	total: 418ms	remaining: 111ms
79:	learn: 0.0029403	total: 424ms	remaining: 106ms
80:	learn: 0.0029172	total: 429ms	remaining: 101ms
81:	learn: 0.0028643	total: 434ms	remaining: 95.4ms
82:	learn: 0.0028016	total: 440ms	remaining: 90.1ms
83:	learn: 0.0027720	total: 445ms	remaining: 84.8ms
84:	learn: 0.0027229	total: 450ms	remaining: 79.4ms
85:	learn: 0.0026711	total: 455ms	remaining: 74.1ms
86:	learn: 0.0026495	total: 460ms	remaining: 68.7ms
87:	learn: 0.0026099	total: 465ms	remaining: 63.4ms
88:	learn: 0.0025740	total: 470ms	remaining: 58.1ms
89:	learn: 0.0025639	total: 475ms	remaining: 52.7ms
90:	learn: 0.0025393	total: 480ms	remaining: 47.5ms
91:	learn: 0.0025147	total: 485ms	remaining: 42.2ms
92:	learn: 0.0025071	total: 490ms	remaining: 36.9ms
93:	learn: 0.0024744	total: 495ms	remaining: 31.6ms
94:	learn: 0.00245

In [20]:
print(precision)
print(recall)

[1.0, 1.0, 1.0, 1.0, 1.0]
[0.6306306306306306, 0.6194690265486725, 0.6, 0.5714285714285714, 0.5538461538461539]


Results for micro avg with 100 estimators:

    precision = [1.0, 1.0, 1.0, 1.0, 1.0]
    
    recall = [0.63, 0.62, 0.6, 0.57, 0.55]

Results for micro avg with 50 estimators:

    precision = [1.0, 1.0, 1.0, 1.0, 1.0]
    
    recall = [0.61, 0.59, 0.57, 0.54, 0.52]

Results for micro avg with 25 estimators:

    precision = [1.0, 1.0, 1.0, 1.0, 1.0]
    
    recall = [0.61, 0.59, 0.56, 0.53, 0.5]

In [21]:
final_pred = model.predict(x_test)

C:\Users\Andre\AppData\Local\Programs\Python\Python310\lib\site-packages\catboost\core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [22]:
final_pred = pd.DataFrame(data=final_pred, columns=TARGET_COLS)

In [23]:
final_pred

,Артериальная гипертензия,ОНМК,"Стенокардия, ИБС, инфаркт миокарда",Сердечная недостаточность,Прочие заболевания сердца
0,0,0,0,1,0
1,1,0,0,1,0
2,1,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
633,1,0,0,0,0
634,1,0,0,0,0
635,1,0,0,0,0
636,0,0,0,0,0


In [24]:
final_pred.to_csv('../data/raw/sample_solution.csv')

Это наши предсказания по валидационной выборке. За неимением y_val проверить их точность, к сожалению, не можем, поэтому просто сохраняем в .csv